In [1]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2 
from torch import nn 

In [14]:
# setup training device
device = "cuda" if torch.cuda.is_available() else "cpu"

# get efficientnet-b0 as a backbone
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
 
# delete the last classifier layer
efficientnet.classifier.fc= nn.Identity()

# freeze all parameters for efficientnet-b0
for param in efficientnet.parameters():
    param.requires_grad = False

# putting model on device 
efficientnet= efficientnet.to(device)

Using cache found in C:\Users\moaaz/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [25]:
# siamese network
class Model(nn.Module): 
    def __init__(self  ): 
        """ 
        model input shape is (batch size , CH, W , H) 
        """
        super().__init__()
        self.pool = nn.Sequential( 
            nn.AvgPool1d(kernel_size = 3, stride=2 , padding=1)
        )
        self.f1 = nn.Sequential( 
            nn.Linear(in_features=640, out_features=1024 ),
            nn.ReLU(),
            nn.Dropout(p=0.2 ),
            nn.BatchNorm1d(num_features=1024)
        ) 
        self.f2 = nn.Sequential( 
            nn.Linear(in_features=1024, out_features=512 ),
            nn.ReLU(),
            nn.Dropout(p=0.2 ),
            nn.BatchNorm1d(num_features=512)
        )
        self.f3= nn.Sequential( 
            nn.Linear(in_features=512, out_features=256 ),
            nn.ReLU(),
            nn.Dropout(p=0.2 ),
            nn.BatchNorm1d(num_features=256))
        self.embed= nn.Sequential( 
            nn.Linear(in_features=256, out_features=128 ),
        ) 
    def forward (self , x ):
        x= efficientnet(x)
        x=self.pool(x)
        x= self.f1(x)
        x= self.f2(x)
        x= self.f3(x)
        x= self.embed(x)
        return x
        
model = Model().to(device)

In [26]:
# our loss calculation class 
class loss_fn(): 
    def __init__(self, margin , model):
        
        self.margin = torch.tensor(margin).to(device)
        self.model = model
        
    def _embedding(self,inputs : list[torch.tensor]):
        """
        inputs : list[torch.tensor] with the shape (positive, anchor, negative)
        output : list[torch.tensor] with the shape (positive, anchor, negative)
        """
        print(inputs.shape)
        
        positive, anchor ,negative = self.model(inputs)

        return [positive,anchor,negative]
        
    def compute_distance(self, inputs):

        embeddings = self._embedding(inputs)
        
        anchorEmbedding = embeddings[1]
        positiveEmbedding = embeddings[0]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = torch.sum( torch.square(anchorEmbedding - positiveEmbedding), axis=-1)
        anDistance = torch.sum( torch.square(anchorEmbedding - negativeEmbedding), axis=-1 )
        
        # return the distances
        return (apDistance, anDistance)
    
    def compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = torch.max(loss + self.margin, torch.tensor(0.0).to(device))
        return loss

In [ ]:
[40 , (3,3,256,256)]

In [27]:
def train_step(model: torch.nn.Module,
               data ,
               loss_fn,
               optimizer,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    
    for batch,inputs in enumerate(data):

        # Send data to GPU
        # postive , anchor , negative = postive.to(device), anchor.to(device), negative.to(device)
        inputs = inputs.to(device)
        apDistance, anDistance = loss_fn.compute_distance(inputs)
        # 2. Calculate loss
        loss = loss_fn.compute_loss(apDistance, anDistance )
        train_loss += loss
        # train_acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels
        # 3. Optimizer zero grad
        optimizer.zero_grad()
        # 4. Loss backward
        loss.backward()
        # 5. Optimizer step
        optimizer.step()
    # Calculate loss and accuracy per epoch and print out what's happening
    # train_loss /= len(data_loader)
    # train_acc /= len(data_loader)
    print(f"Train loss: {train_loss}")

In [98]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [99]:
# random sample 
sample = torch.randn((3,3,256,256))

In [100]:
loss_function = loss_fn(0.1 , model)

In [176]:
train_step(model, [sample], loss_function, optimizer,  device)

torch.Size([3, 3, 256, 256])
Train loss: 0.0
